# Introduction
Searching for specific scenes in a movie can be time-consuming. By combining video processing, AI-generated captions, and embeddings, we can create a powerful tool that allows users to find scenes based on textual descriptions. This tool extracts key frames from a video, generates captions for each frame, computes embeddings, and allows users to search through the scenes using natural language queries.

## Installing Required Libraries
Install the necessary libraries

In [ ]:
%pip install openai
%pip install opencv-python
%pip install scikit-learn
%pip install scenedetect
%pip install PySceneDetect
%pip install boto3
%pip install pandas
%pip install python-dotenv

# Import the necessary modules

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import boto3
from sklearn.metrics.pairwise import cosine_similarity
from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector
from IPython.display import Image, display
import concurrent.futures
from dotenv import load_dotenv

/Users/bbassett/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load the environment variables

In [3]:
load_dotenv()

True

# Initializing OpenAI Client
Initialize the OpenAI client using your API key:

In [4]:
from openai import OpenAI
# Initializing OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# AWS S3 Configuration
Set up your AWS S3 client:

In [5]:
# AWS Credentials
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

# AWS S3 Configuration
s3_bucket_name = 'video-clips-test'
s3_region = 'us-east-2'

# Initialize S3 client
s3 = boto3.client('s3',
                  region_name=s3_region,
                  aws_access_key_id=aws_access_key_id,
                  aws_secret_access_key=aws_secret_access_key)

Replace 'your-s3-bucket-name' and 'your-s3-region' with your AWS S3 bucket name and region.

# Downloading Video from S3
We'll start by downloading the video file from AWS S3:

In [6]:
# Function to download video from S3
def download_video_from_s3(s3_bucket, s3_key, local_path):
    s3.download_file(s3_bucket, s3_key, local_path)
    print(f"Downloaded {s3_key} from S3 to {local_path}")

Call the function with the appropriate parameters:

In [7]:
# Video S3 key (path within the bucket)
s3_video_key = 'movie.mp4'  # Update if your video is stored under a different key

# Local path to save the video
video_local_path = 'movie.mp4'  # Save in the current working directory

# Download the video from S3
download_video_from_s3(s3_bucket_name, s3_video_key, video_local_path)

Downloaded movie.mp4 from S3 to movie.mp4


# Extracting Key Frames from Video
Next, we'll extract key frames from the video using PySceneDetect:

In [8]:
# Function to extract key frames from video
def extract_key_frames(video_path, output_dir):
    import os
    import cv2
    from scenedetect import VideoManager, SceneManager
    from scenedetect.detectors import ContentDetector

    # Create a VideoManager object
    video_manager = VideoManager([video_path])
    # Create a SceneManager object
    scene_manager = SceneManager()
    # Add ContentDetector algorithm (detects fast cuts)
    scene_manager.add_detector(ContentDetector(threshold=30.0))

    # Start the video manager and perform scene detection
    try:
        video_manager.start()
        scene_manager.detect_scenes(frame_source=video_manager)
    except Exception as e:
        print(f"Error during scene detection: {e}")
        return []

    # Get list of detected scenes
    scene_list = scene_manager.get_scene_list()
    if not scene_list:
        print("No scenes were detected in the video.")
        return []
    else:
        print(f"Detected {len(scene_list)} scenes in video.")

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Open video file using OpenCV
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Failed to open video file {video_path}")
        return []

    # Save the first frame of each scene
    frame_list = []
    total_scenes = len(scene_list)
    for i, scene in enumerate(scene_list):
        print(f"Processing scene {i + 1}/{total_scenes}")
        # Access the start timecode of the scene
        start_timecode = scene[0]
        # Get the frame number
        start_frame = start_timecode.get_frames()
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        ret, frame = cap.read()
        if ret:
            frame_filename = os.path.join(output_dir, f"frame_{i}.jpg")
            cv2.imwrite(frame_filename, frame)
            frame_list.append(frame_filename)
        else:
            print(f"Failed to read frame at position {start_frame}")
    cap.release()
    video_manager.release()  # Release the video manager resources
    return frame_list

Extract the frames:

In [9]:
# Output directory for frames
output_dir = 'frames'  # Local directory to save frames

# Extract key frames from the video
frame_list = extract_key_frames(video_local_path, output_dir)

VideoManager is deprecated and will be removed.


Detected 308 scenes in video.
Processing scene 1/308
Processing scene 2/308
Processing scene 3/308
Processing scene 4/308
Processing scene 5/308
Processing scene 6/308
Processing scene 7/308
Processing scene 8/308
Processing scene 9/308
Processing scene 10/308
Processing scene 11/308
Processing scene 12/308
Processing scene 13/308
Processing scene 14/308
Processing scene 15/308
Processing scene 16/308
Processing scene 17/308
Processing scene 18/308
Processing scene 19/308
Processing scene 20/308
Processing scene 21/308
Processing scene 22/308
Processing scene 23/308
Processing scene 24/308
Processing scene 25/308
Processing scene 26/308
Processing scene 27/308
Processing scene 28/308
Processing scene 29/308
Processing scene 30/308
Processing scene 31/308
Processing scene 32/308
Processing scene 33/308
Processing scene 34/308
Processing scene 35/308
Processing scene 36/308
Processing scene 37/308
Processing scene 38/308
Processing scene 39/308
Processing scene 40/308
Processing scene 41

# Uploading Images to S3
We'll upload the extracted frames to S3 and get their public URLs:

In [10]:
# Function to upload image to S3 and get public URL
def upload_image_to_s3(image_path, s3_bucket, s3_folder='frames'):
    file_name = os.path.basename(image_path)
    s3_key = f"{s3_folder}/{file_name}"
    s3.upload_file(image_path, s3_bucket, s3_key)
    image_url = f"https://{s3_bucket}.s3.{s3_region}.amazonaws.com/{s3_key}"
    return image_url

# Generating Captions with OpenAI
We'll generate captions for each frame using OpenAI's GPT model:

In [11]:
# Function to generate captions using OpenAI's gpt-4o-mini model
def generate_caption(image_url):
    caption_system_prompt = '''
    You are an assistant that generates concise captions for images. These captions will be embedded and stored so 
    people can semantically search for scenes. Ensure your captions include:
    - Physical descriptions of people
    - Identify and name of key actors
    - Descriptions of key scene objects such as their color
    - The mood of the scene
    - The actions or activites taking place in the scene
    '''

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": caption_system_prompt
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": image_url,
                        }
                    }
                ],
            }
        ],
        max_tokens=300,
        temperature=0.2
    )
    return response.choices[0].message.content

# Getting Embeddings
Compute embeddings for the captions using OpenAI's embedding models:

In [12]:
# Function to get embeddings
def get_embedding(value, model="text-embedding-3-large"): 
    embeddings = client.embeddings.create(
        model=model,
        input=value,
        encoding_format="float"
    )
    return embeddings.data[0].embedding

# Searching Frames Based on a Prompt
We'll search for frames that are most similar to a user's query:

In [13]:
# Function to search frames based on a prompt
def search_frames(prompt, df_frames, top_n=5):
    prompt_embedding = get_embedding(prompt)
    df_frames['similarity'] = df_frames['embedding'].apply(
        lambda x: cosine_similarity(np.array(x).reshape(1, -1), np.array(prompt_embedding).reshape(1, -1))[0][0]
    )
    results = df_frames.sort_values('similarity', ascending=False).head(top_n)
    return results

# Putting It All Together
We'll process each frame, generate captions and embeddings, and store the data in a DataFrame:

In [14]:
# Function to process a single frame
def process_frame(frame_filename):
    # Upload image to S3 and get the URL
    image_url = upload_image_to_s3(frame_filename, s3_bucket_name)
    # Generate caption
    caption = generate_caption(image_url)
    # Generate embedding
    embedding = get_embedding(caption)
    # Return frame data
    return {
        'frame_filename': frame_filename,
        'image_url': image_url,
        'caption': caption,
        'embedding': embedding
    }

# Process frames in parallel
frame_data_list = []
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(process_frame, frame_list)
    frame_data_list = list(results)

# Create DataFrame
df_frames = pd.DataFrame(frame_data_list)

# User Query and Displaying Results
Now, let's allow the user to search for scenes:

In [21]:
prompt = "Find images of Paul Walker, a blonde hair blue-eyed man, in a car."
results = search_frames(prompt, df_frames, top_n=5)

# Display the results
for index, row in results.iterrows():
    print(f"Frame: {row['frame_filename']}")
    print(f"Caption: {row['caption']}")
    print(f"Similarity: {row['similarity']}")
    display(Image(url=row['image_url']))

Frame: frames/frame_183.jpg
Caption: A close-up of a male driver with short, light brown hair and intense blue eyes, looking focused and serious while seated in a car. The interior is dimly lit, with a blue hue illuminating his face, suggesting a tense or dramatic mood. The background is blurred, indicating high speed or movement.
Similarity: 0.5480666149492754


Frame: frames/frame_21.jpg
Caption: A close-up shot of a young man with short, light brown hair and striking blue eyes, sitting in a car. The scene is illuminated with a blue hue, creating a tense and focused atmosphere. The man appears serious and contemplative, possibly preparing for a race or a significant moment. The interior of the car is visible, with racing seats in the background.
Similarity: 0.5474135337296182


Frame: frames/frame_193.jpg
Caption: A man with short, light-colored hair is driving a car, wearing a white shirt. The interior of the vehicle is illuminated with blue lights, creating a dynamic and intense atmosphere. The scene conveys a sense of speed and excitement, as the car appears to be in motion.
Similarity: 0.510289437589817


Frame: frames/frame_156.jpg
Caption: A tense scene inside a car featuring a man with short, light brown hair and a serious expression. He is wearing a white shirt and a racing harness. The interior is dimly lit with blue tones, suggesting a high-stakes moment. The background hints at a nighttime setting with colorful lights, indicating a racing or street racing atmosphere.
Similarity: 0.4919880567964413


Frame: frames/frame_120.jpg
Caption: A young man with short, light brown hair is seated in a car, wearing a white t-shirt and a racing harness. The interior is dimly lit with blue tones, creating a tense atmosphere. He appears focused and slightly tense, gripping the steering wheel as if preparing for action. The background suggests a nighttime setting, possibly on a street or racetrack.
Similarity: 0.48366245830524235
